# Side-by-side image comparison
We have a set of icons.
We need to find out which icon people prefer and determine the top icon out of the set.
We ask performers to look at the icons and choose the one they prefer and then we aggregate these results to obtain the top icon.

<img src="https://tlk.s3.yandex.net/dataset/toloka_logos/b2b_blossom_icon.png" width="200"/>
<img src="https://tlk.s3.yandex.net/dataset/toloka_logos/b2b_dandelion_icon.png" width="200"/>
<img src="https://tlk.s3.yandex.net/dataset/toloka_logos/b2b_sunrise_icon.png" width="200"/>
<img src="https://tlk.s3.yandex.net/dataset/toloka_logos/b2c_blossom_icon.png" width="200"/>
<img src="https://tlk.s3.yandex.net/dataset/toloka_logos/b2c_dandelion_icon.png" width="200"/>
<img src="https://tlk.s3.yandex.net/dataset/toloka_logos/b2c_sunrise_icon.png" width="200"/>

### Call to action
If you found some bugs or have a new feature idea, don't hesitate to [open a new issue on Github](https://github.com/Toloka/toloka-kit/issues/new/choose).
Like our library and examples? Star [our repo on Github](https://github.com/Toloka/toloka-kit)

Prepare environment and import all we'll need.

In [ ]:
%%capture
!pip install toloka-kit==0.1.26
!pip install crowd-kit==1.0.0
!pip install pandas
!pip install ipyplot

import datetime
import itertools
import sys
import time
import logging
import getpass

import ipyplot
import pandas

import toloka.client as toloka
import toloka.client.project.template_builder as tb
from crowdkit.aggregation import NoisyBradleyTerry

In [ ]:
logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)

Сreate toloka-client instance. All api calls will go through it. More about OAuth token in our [Learn the basics example](https://github.com/Toloka/toloka-kit/tree/main/examples/0.getting_started/0.learn_the_basics) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/0.getting_started/0.learn_the_basics/learn_the_basics.ipynb)

In [ ]:
toloka_client = toloka.TolokaClient(getpass.getpass('Enter your OAuth token: '), 'PRODUCTION') # Or switch to 'SANDBOX'
print(toloka_client.get_requester())

## Create a project

In [ ]:
project = toloka.Project(
    assignments_issuing_type='AUTOMATED',
    public_name='Which icon do you like more?',
    public_description='Look at the icons and decide which one you like more.',
)

Create task interface

In [ ]:
main_interface = tb.SideBySideLayoutV1(
    items=[
        tb.ImageViewV1(url=tb.InputData(path='image_left'), full_height=True),
        tb.ImageViewV1(url=tb.InputData(path='image_right'), full_height=True),
    ],
    controls=tb.ButtonRadioGroupFieldV1(
        data=tb.OutputData(path='result'),
        label='Which icon do you like more?',
        options=[
            tb.GroupFieldOption(label='Left', value='LEFT'),
            tb.GroupFieldOption(label='Right', value='RIGHT'),
            tb.GroupFieldOption(label='Loading error', value='ERROR'),
        ]
    )
)

hot_keys_plugin = tb.HotkeysPluginV1(
    key_0=tb.SetActionV1(data=tb.OutputData(path='result'), payload='ERROR'),
    key_1=tb.SetActionV1(data=tb.OutputData(path='result'), payload='LEFT'),
    key_2=tb.SetActionV1(data=tb.OutputData(path='result'), payload='RIGHT'),
)

project_interface = toloka.project.view_spec.TemplateBuilderViewSpec(
    config=tb.TemplateBuilder(
        view=main_interface,
        plugins=[hot_keys_plugin],
    )
)

Set data specification. And set task interface to project.

In [ ]:
input_specification = {
    'image_left': toloka.project.field_spec.UrlSpec(),
    'image_right': toloka.project.field_spec.UrlSpec(),
}
output_specification = {'result': toloka.project.field_spec.StringSpec()}

project.task_spec = toloka.project.task_spec.TaskSpec(
    input_spec=input_specification,
    output_spec=output_specification,
    view_spec=project_interface,
)

Write short and simple 	instructions.

In [ ]:
project.public_instructions = """<p>Look at the icons and decide which one you like more.</p>
<p>Select "<b>Left</b>" if you like the icon on the left more.</p>
<p>Select "<b>Right</b>" if you like the icon on the right more.</p>
<p>Select "<b>Loadinf error</b>" if the picture failed to load.</p>"""

Create a project.

In [ ]:
project = toloka_client.create_project(project)

The performer will see the interface like this:

<table  align="center">
  <tr>
  <td>
    <img src="./img/performer_interface.png"
         alt="Photo examples"  width="800">
  </td></tr>
  <tr>
  <td align="center">
    <b>Figure 1.</b> How performer will see your task
  </td></tr>
</table>

## Create a pool
Specify the [pool parameters.](https://toloka.ai/en/docs/guide/concepts/pool_poolparams?utm_source=github&utm_medium=site&utm_campaign=tolokakit)

In [ ]:
pool = toloka.Pool(
    project_id=project.id,
    # Give the pool any convenient name. You are the only one who will see it.
    private_name='Which icon do you like more',
    may_contain_adult_content=False,
    # Set the price per task page.
    reward_per_assignment=0.01,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    # Time given to complete a task suite
    assignment_max_duration_seconds=600,
)

Select English-speaking performers

In [ ]:
pool.filter = toloka.filter.Languages.in_('EN')

Set up [Quality control](https://toloka.ai/en/docs/guide/concepts/control?utm_source=github&utm_medium=site&utm_campaign=tolokakit). Set up the Submitted responses quality control rule. Restrict the number of responses per user to one. This way you will only get one answer from each user and thus ensure a variety of opinions.

In [ ]:
pool.quality_control.add_action(
    collector=toloka.collectors.AnswerCount(),
    conditions=[toloka.conditions.AssignmentsAcceptedCount == 1],
    action=toloka.actions.RestrictionV2(
        scope=toloka.user_restriction.UserRestriction.PROJECT,
        duration=3,
        duration_unit='DAYS',
        private_comment='No need more answers from this performer',
    )
)

Overlap. This is the number of users who will complete the same task. Since you are interested in a variety of opinions, select a big overlap for each task. For example, 10.

In [ ]:
pool.defaults = toloka.Pool.Defaults(default_overlap_for_new_task_suites=10)

Specify	the number of tasks per page. 1 task per page. A performer will only see one pair of images on a page.

In [ ]:
pool.set_mixer_config(
    real_tasks_count=1,
    golden_tasks_count=0,
    training_tasks_count=0
)

Create pool

In [ ]:
pool = toloka_client.create_pool(pool)

## Preparing and uploading tasks

This example uses a small data set with images.

The dataset used is collected by Toloka team and distributed under a Creative Commons Attribution 4.0 International license
[![License: CC BY 4.0](https://img.shields.io/badge/License-CC%20BY%204.0-lightgrey.svg)](https://creativecommons.org/licenses/by/4.0/).

In [ ]:
!curl https://tlk.s3.yandex.net/dataset/toloka_logos/toloka_logos.tsv --output dataset.tsv
dataset = pandas.read_csv('dataset.tsv', sep='\t')
with pandas.option_context("max_colwidth", 80):
    print(dataset)

Our project is a pairwise comparison of two images. But our dataset contains just a flat list. Let's create a dataset that contains all possible pairs for comparison.

In [ ]:
dataset = pandas.DataFrame(itertools.combinations(dataset['url'], 2), columns=['image_left', 'image_right'])
with pandas.option_context("max_colwidth", 70):
    display(dataset)

Create pool tasks

In [ ]:
tasks = [
    toloka.Task(
        pool_id=pool.id,
        input_values={
            'image_left': row['image_left'],
            'image_right': row['image_right'],
        }
    )
    for i, row in dataset.iterrows()
]

Upload tasks

In [ ]:
created_tasks = toloka_client.create_tasks(tasks, allow_defaults=True)
print(len(created_tasks.items))

Start the pool.

**Important.** Remember that real Toloka performers will complete the tasks.
Double check that everything is correct
with your project configuration before you start the pool

In [ ]:
pool = toloka_client.open_pool(pool.id)
print(pool.status)

## Receiving responses

Wait until the pool is completed.

In [ ]:
pool_id = pool.id

def wait_pool_for_close(pool_id, minutes_to_wait=1):
    sleep_time = 60 * minutes_to_wait
    pool = toloka_client.get_pool(pool_id)
    while not pool.is_closed():
        op = toloka_client.get_analytics([toloka.analytics_request.CompletionPercentagePoolAnalytics(subject_id=pool.id)])
        op = toloka_client.wait_operation(op)
        percentage = op.details['value'][0]['result']['value']
        print(
            f'   {datetime.datetime.now().strftime("%H:%M:%S")}\t'
            f'Pool {pool.id} - {percentage}%'
        )
        time.sleep(sleep_time)
        pool = toloka_client.get_pool(pool.id)
    print('Pool was closed.')

wait_pool_for_close(pool_id)

Get responses

When all the tasks are completed, look at the responses from performers.

In [ ]:
answers = []

for assignment in toloka_client.get_assignments(pool_id=pool_id, status='ACCEPTED'):
    for task, solution in zip(assignment.tasks, assignment.solutions):
        answers.append(
            [
                task.input_values['image_left'],
                task.input_values['image_right'],
                solution.output_values['result'],
                assignment.user_id
            ]
        )

print(f'answers count: {len(answers)}')

Ranking after a pairwise comparison is quite a difficult task. We will use the Bradley-Terry algorithm, which is already implemented in the Crowd-Kit and allows you to get the result in a few lines of code.

> David R. Hunter. 2004.
> MM algorithms for generalized Bradley-Terry models
> Ann. Statist., Vol. 32, 1 (2004): 384–406.
>
>
> Bradley, R. A. and Terry, M. E. 1952.
> Rank analysis of incomplete block designs. I. The method of paired comparisons.
> Biometrika, Vol. 39 (1952): 324–345.

In [ ]:
# Prepare dataframe
answers_df = pandas.DataFrame(answers, columns=['left', 'right', 'label', 'worker'])

answers_df = answers_df[(answers_df.label == 'LEFT') | (answers_df.label == 'RIGHT')]
answers_df['label'] = answers_df.apply(lambda row: row[row['label'].lower()], axis=1)

# Run aggregation
result = NoisyBradleyTerry().fit_predict(answers_df).sort_values(ascending=False)
print(result)

Let's look at the ranking results

In [ ]:
images = result.index.values
labels = result.values
ipyplot.plot_images(
    images=images,
    labels=labels,
    max_images=6,
    img_width=200,
)

**You** can see the ranked images. Some possible results are shown in figure 2 below.

<table  align="center">
  <tr><td>
    <img src="./img/possible_results.png"
         alt="Possible results">
  </td></tr>
  <tr><td align="center">
    <b>Figure 2.</b> Possible results.
  </td></tr>
</table>